In [39]:
import ibis
from boring_semantic_layer.semantic_api.api import to_semantic_table

con = ibis.duckdb.connect("test.duckdb")

BASE_URL = "https://pub-a45a6a332b4646f2a6f44775695c64df.r2.dev"
tables = {
    "flights_tbl": con.read_parquet(f"{BASE_URL}/flights.parquet"),
    "carriers_tbl": con.read_parquet(f"{BASE_URL}/carriers.parquet"),
}


In [4]:
tables["flights_tbl"]

DatabaseTable: ibis_read_parquet_ymeri6hsyraczeezmkfazpfg2e
  carrier     string
  origin      string
  destination string
  flight_num  string
  flight_time int64
  tail_num    string
  dep_time    timestamp(6)
  arr_time    timestamp(6)
  dep_delay   int64
  arr_delay   int64
  taxi_out    int64
  taxi_in     int64
  distance    int64
  cancelled   string
  diverted    string
  id2         int64

In [6]:
flights_st = (
    to_semantic_table(tables["flights_tbl"])
    .with_dimensions(
        origin=lambda t: t.origin,
        carrier=lambda t: t.carrier,
        month=lambda t: t.arr_time.truncate("month"),
    )
    .with_measures(
        flight_count=lambda t: t.count(),
        avg_dep_delay=lambda t: t.dep_delay.mean(),
    )
)

In [12]:
flights_st.group_by("carrier").aggregate(lambda t: t.flight_count).execute()

,carrier,flight_count
0,B6,4842
1,AS,8453
2,HP,9750
3,US,37683
4,OH,4420
5,MQ,15869
6,AA,34577
7,EV,15769
8,UA,32757
9,RU,16074


In [ ]:
import malloy
from malloy.data.duckdb import DuckDbConnection
BASE_URL = "https://pub-a45a6a332b4646f2a6f44775695c64df.r2.dev"
tables = {
    "flights_tbl": con.read_parquet(f"{BASE_URL}/flights.parquet"),
    "carriers_tbl": con.read_parquet(f"{BASE_URL}/carriers.parquet"),
    "airports_tbl": con.read_parquet(f"{BASE_URL}/airports.parquet"),
}

with malloy.Runtime() as runtime:
    runtime.add_connection(DuckDbConnection(home_dir="."))

    data = await runtime.load_source(
        """
        source:airports is duckdb.table('%s/airports.parquet') extend{
            primary_key: code
            dimension: name is concat(code, ' - ', full_name)
            dimension: faa_region_name is faa_region ?
                pick 'Southwest' when 'ASW'
                pick 'Northwest Mountain' when 'ANM'
                pick 'Eastern' when 'AEA'
                pick 'Southern' when 'ASO'
                pick 'Great Lakes' when 'AGL'
                pick 'Central' when 'ACE'
                pick 'New England' when 'ANE'
                pick 'Western Pacific' when 'AWP'
                pick 'Alaska' when 'AAL'

            view: by_region is {
                group_by: faa_region
                group_by: faa_region_name
                aggregate: airport_count
            }

            view: by_state is {
                where: state is not null
                group_by: state
                aggregate: airport_count
            }
        }
        """ % BASE_URL  
    ).run(query="""
        run: airports -> {
          group_by: code
          aggregate: code_count is count()
          limit: 5
        }
    """)
    df = data.to_dataframe()
    print(df)



I0000 00:00:1756285255.762703 65473345 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


MalloyRuntimeError: error: no viable alternative at input '{where:stateis' at line 23